In [2]:
# -*- coding: utf-8 -*-
"""Unified Model Comparison: GAP, TPA, Gated-TPA
"""

from google.colab import drive
drive.mount('/content/drive')

import os, random, time, copy, json
import numpy as np
from typing import Tuple, Dict, List
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

# ========================
# Config & Reproducibility
# ========================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

@dataclass
class Config:
    data_dir: str = "/content/drive/MyDrive/AI_data/TPA2/pamap2_transition_datasets"
    save_dir: str = "/content/drive/MyDrive/AI_data/TPA2"

    epochs: int = 100
    batch_size: int = 128
    lr: float = 1e-4
    weight_decay: float = 1e-4
    grad_clip: float = 1.0
    label_smoothing: float = 0.05

    patience: int = 20
    min_delta: float = 0.0001
    val_split: float = 0.2

    d_model: int = 128

    # TPA hyperparameters
    tpa_num_prototypes: int = 16
    tpa_heads: int = 4
    tpa_dropout: float = 0.1
    tpa_temperature: float = 0.07
    tpa_topk_ratio: float = 0.25

    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    num_workers: int = 2

cfg = Config()

# ========================
# Dataset Class
# ========================
class PreloadedDataset(Dataset):
    """Dataset for pre-loaded numpy arrays"""
    def __init__(self, X: np.ndarray, y: np.ndarray):
        super().__init__()
        self.X = torch.from_numpy(X).float()

        # Label 범위 확인 및 조정 (1-6 -> 0-5)
        if y.min() >= 1:
            y = y - 1

        self.y = torch.from_numpy(y).long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ========================
# Data Loading Functions
# ========================
def load_dataset(base_dir: str, dataset_name: str):
    """
    Load pre-augmented dataset
    Args:
        base_dir: base directory containing all datasets
        dataset_name: e.g., "ORIGINAL", "STANDING_TO_SITTING_10pct", etc.
    Returns:
        train_dataset, test_dataset
    """
    dataset_dir = os.path.join(base_dir, dataset_name)

    print(f"\nLoading {dataset_name}...")
    print(f"  Path: {dataset_dir}")

    # Load data
    X_train = np.load(os.path.join(dataset_dir, "X_train.npy"))
    y_train = np.load(os.path.join(dataset_dir, "y_train.npy"))
    X_test = np.load(os.path.join(dataset_dir, "X_test.npy"))
    y_test = np.load(os.path.join(dataset_dir, "y_test.npy"))

    print(f"  Train: {X_train.shape}, Test: {X_test.shape}")

    train_dataset = PreloadedDataset(X_train, y_train)
    test_dataset = PreloadedDataset(X_test, y_test)

    return train_dataset, test_dataset

# ========================
# Model Components
# ========================
class ConvBNAct(nn.Module):
    def __init__(self, c_in, c_out, k, s=1, p=None, g=1):
        super().__init__()
        self.c = nn.Conv1d(c_in, c_out, k, s, k//2 if p is None else p, groups=g, bias=False)
        self.bn = nn.BatchNorm1d(c_out)
        self.act = nn.GELU()

    def forward(self, x):
        return self.act(self.bn(self.c(x)))

class MultiPathCNN(nn.Module):
    """Shared backbone for all models"""
    def __init__(self, in_ch=27, d_model=128, branches=(3,5,9,15), stride=2):
        super().__init__()
        h = d_model // 2
        self.pre = ConvBNAct(in_ch, h, 1)
        self.branches = nn.ModuleList([
            nn.Sequential(ConvBNAct(h, h, k, stride, g=h), ConvBNAct(h, h, 1))
            for k in branches
        ])
        self.post = ConvBNAct(len(branches)*h, d_model, 1)

    def forward(self, x):
        return self.post(torch.cat([b(self.pre(x)) for b in self.branches], dim=1))

# ========================
# GAP Model
# ========================
class GAPModel(nn.Module):
    """Baseline: Global Average Pooling"""
    def __init__(self, d_model=128, num_classes=12):
        super().__init__()
        self.backbone = MultiPathCNN(d_model=d_model)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        # x shape: [B, T, C] -> transpose to [B, C, T] for Conv1d
        x = x.transpose(1, 2)  # [B, C, T]
        fmap = self.backbone(x)  # [B, D, T]
        features = fmap.transpose(1, 2)  # [B, T, D]
        pooled = features.mean(dim=1)  # [B, D]
        logits = self.fc(pooled)
        return logits

# ========================
# Pure-TPA
# ========================
class ProductionTPA(nn.Module):
    """Pure TPA"""
    def __init__(self, dim, num_prototypes=16, heads=4, dropout=0.1,
                 temperature=0.07, topk_ratio=0.25):
        super().__init__()
        assert dim % heads == 0

        self.dim = dim
        self.heads = heads
        self.head_dim = dim // heads
        self.num_prototypes = num_prototypes
        self.temperature = temperature
        self.topk_ratio = topk_ratio

        self.proto = nn.Parameter(torch.randn(num_prototypes, dim) * 0.02)

        self.pre_norm = nn.LayerNorm(dim)

        self.q_proj = nn.Linear(dim, dim, bias=False)
        self.k_proj = nn.Linear(dim, dim, bias=False)
        self.v_proj = nn.Linear(dim, dim, bias=False)

        self.fuse = nn.Sequential(
            nn.Linear(dim, dim),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(dim, dim)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, D = x.shape
        P = self.num_prototypes

        x_norm = self.pre_norm(x)

        K = self.k_proj(x_norm)
        V = self.v_proj(x_norm)
        Qp = self.q_proj(self.proto).unsqueeze(0).expand(B, -1, -1)

        def split_heads(t, length):
            return t.view(B, length, self.heads, self.head_dim).transpose(1, 2)

        Qh = split_heads(Qp, P)
        Kh = split_heads(K, T)
        Vh = split_heads(V, T)

        Qh = F.normalize(Qh, dim=-1)
        Kh = F.normalize(Kh, dim=-1)

        scores = torch.matmul(Qh, Kh.transpose(-2, -1)) / self.temperature
        attn = F.softmax(scores, dim=-1)
        attn = torch.nan_to_num(attn, nan=0.0)
        attn = self.dropout(attn)

        proto_tokens = torch.matmul(attn, Vh)
        proto_tokens = proto_tokens.transpose(1, 2).contiguous().view(B, P, D)

        z_tpa = proto_tokens.mean(dim=1)

        z = self.fuse(z_tpa)

        return z

class TPAModel(nn.Module):
    def __init__(self, d_model=128, num_classes=12, tpa_config=None):
        super().__init__()
        self.backbone = MultiPathCNN(d_model=d_model)
        self.tpa = ProductionTPA(
            dim=d_model,
            num_prototypes=tpa_config['num_prototypes'],
            heads=tpa_config['heads'],
            dropout=tpa_config['dropout'],
            temperature=tpa_config['temperature'],
            topk_ratio=tpa_config['topk_ratio']
        )
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        # x shape: [B, T, C] -> transpose to [B, C, T] for Conv1d
        x = x.transpose(1, 2)  # [B, C, T]
        fmap = self.backbone(x)  # [B, D, T]
        features = fmap.transpose(1, 2)  # [B, T, D]
        z = self.tpa(features)  # [B, D]
        logits = self.classifier(z)
        return logits

# ========================
# Gated-TPA
# ========================
class GatedTPAModel(nn.Module):
    def __init__(self, d_model=128, num_classes=12, tpa_config=None):
        super().__init__()
        self.backbone = MultiPathCNN(d_model=d_model)
        self.tpa = ProductionTPA(
            dim=d_model,
            num_prototypes=tpa_config['num_prototypes'],
            heads=tpa_config['heads'],
            dropout=tpa_config['dropout'],
            temperature=tpa_config['temperature'],
            topk_ratio=tpa_config['topk_ratio']
        )

        # Gating mechanism
        self.gate = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.Sigmoid()
        )

        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        # x shape: [B, T, C] -> transpose to [B, C, T] for Conv1d
        x = x.transpose(1, 2)  # [B, C, T]
        fmap = self.backbone(x)  # [B, D, T]
        features = fmap.transpose(1, 2)  # [B, T, D]

        # GAP branch
        z_gap = features.mean(dim=1)  # [B, D]

        # TPA branch
        z_tpa = self.tpa(features)  # [B, D]

        # Gating
        gate_input = torch.cat([z_gap, z_tpa], dim=-1)
        g = self.gate(gate_input)

        # Gated fusion
        z = g * z_gap + (1 - g) * z_tpa

        logits = self.classifier(z)
        return logits

# ========================
# Training & Evaluation
# ========================
def train_one_epoch(model, loader, opt, cfg: Config):
    model.train()
    total, correct, loss_sum = 0, 0, 0.0

    for x, y in loader:
        x, y = x.to(cfg.device).float(), y.to(cfg.device)

        opt.zero_grad(set_to_none=True)
        logits = model(x)

        loss = F.cross_entropy(logits, y, label_smoothing=cfg.label_smoothing)
        if torch.isnan(loss):
            continue

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)
        opt.step()

        with torch.no_grad():
            pred = logits.argmax(dim=-1)
            correct += (pred == y).sum().item()
            total += y.size(0)
            loss_sum += loss.item() * y.size(0)

    return {
        "loss": loss_sum / total if total > 0 else 0,
        "acc": correct / total if total > 0 else 0
    }

@torch.no_grad()
def evaluate(model, loader, cfg: Config):
    model.eval()
    ys, ps = [], []

    for x, y in loader:
        x, y = x.to(cfg.device), y.to(cfg.device)
        logits = model(x)
        ps.append(logits.argmax(dim=-1).cpu().numpy())
        ys.append(y.cpu().numpy())

    y_true, y_pred = np.concatenate(ys), np.concatenate(ps)
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')

    return acc, f1

def train_model(model, train_loader, val_loader, cfg: Config, model_name: str):
    """Train a single model"""
    print(f"\n[Training {model_name}]")

    opt = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)

    best_acc, best_wts = 0.0, None
    patience_counter = 0

    for epoch in range(1, cfg.epochs + 1):
        stats = train_one_epoch(model, train_loader, opt, cfg)
        val_acc, val_f1 = evaluate(model, val_loader, cfg)

        if val_acc > best_acc + cfg.min_delta:
            best_acc = val_acc
            best_wts = copy.deepcopy(model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1

        if epoch % 10 == 0:
            print(f"  Epoch {epoch:3d}: Train Acc={stats['acc']:.4f}, Val Acc={val_acc:.4f}, F1={val_f1:.4f}")

        if patience_counter >= cfg.patience:
            print(f"  Early stopping at epoch {epoch}")
            break

    if best_wts:
        model.load_state_dict(best_wts)

    print(f"  Best Val Acc: {best_acc:.4f}")
    return best_acc

def create_model(model_name: str, cfg: Config):
    """Create model by name"""
    tpa_config = {
        'num_prototypes': cfg.tpa_num_prototypes,
        'heads': cfg.tpa_heads,
        'dropout': cfg.tpa_dropout,
        'temperature': cfg.tpa_temperature,
        'topk_ratio': cfg.tpa_topk_ratio
    }

    if model_name == "GAP":
        return GAPModel(d_model=cfg.d_model).to(cfg.device).float()
    elif model_name == "TPA":
        return TPAModel(d_model=cfg.d_model, tpa_config=tpa_config).to(cfg.device).float()
    elif model_name == "Gated-TPA":
        return GatedTPAModel(d_model=cfg.d_model, tpa_config=tpa_config).to(cfg.device).float()
    else:
        raise ValueError(f"Unknown model: {model_name}")

# ========================
# Main Experiment
# ========================
def run_experiment(dataset_name: str, cfg: Config):
    """Run complete experiment for one dataset"""

    print(f"\n{'='*80}")
    print(f"EXPERIMENT: {dataset_name}")
    print(f"{'='*80}")

    # Load data
    train_dataset, test_dataset = load_dataset(cfg.data_dir, dataset_name)

    # Split train into train/val using indices
    n_total = len(train_dataset)
    indices = np.arange(n_total)

    # Get labels for stratification
    y_labels = train_dataset.y.numpy()

    train_indices, val_indices = train_test_split(
        indices,
        test_size=cfg.val_split,
        random_state=SEED,
        stratify=y_labels
    )

    # Create subsets using Subset
    from torch.utils.data import Subset
    train_subset = Subset(train_dataset, train_indices)
    val_subset = Subset(train_dataset, val_indices)

    # Create data loaders
    g = torch.Generator(device='cpu').manual_seed(SEED)
    train_loader = DataLoader(train_subset, cfg.batch_size, shuffle=True,
                              num_workers=cfg.num_workers, generator=g)
    val_loader = DataLoader(val_subset, cfg.batch_size, num_workers=cfg.num_workers)
    test_loader = DataLoader(test_dataset, cfg.batch_size, num_workers=cfg.num_workers)

    print(f"\nDataset splits:")
    print(f"  Train: {len(train_subset)}, Val: {len(val_subset)}, Test: {len(test_dataset)}")

    # Train and evaluate all models
    results = []
    model_names = ["GAP", "TPA", "Gated-TPA"]

    for model_name in model_names:
        # Reset seed for each model
        random.seed(SEED)
        np.random.seed(SEED)
        torch.manual_seed(SEED)

        # Create and train model
        model = create_model(model_name, cfg)
        best_val_acc = train_model(model, train_loader, val_loader, cfg, model_name)

        # Evaluate on test set
        test_acc, test_f1 = evaluate(model, test_loader, cfg)

        print(f"\n[{model_name} Results]")
        print(f"  Val Acc: {best_val_acc:.4f}")
        print(f"  Test Acc: {test_acc:.4f}, F1: {test_f1:.4f}")

        results.append({
            'Model': model_name,
            'Dataset': dataset_name,
            'Val_Accuracy': float(best_val_acc),
            'Test_Accuracy': float(test_acc),
            'Test_F1_Score': float(test_f1)
        })

    return results

# ========================
# Run All Experiments
# ========================
if __name__ == "__main__":
    print("\n" + "="*80)
    print("UNIFIED MODEL COMPARISON: GAP vs TPA vs Gated-TPA")
    print("="*80)

    datasets = []

    transitions = [
        'Standing_TO_Walking',
        'Walking_TO_Standing',
        'Walking_TO_Running',
        'Running_TO_Walking',
        'Walking_TO_Ascending_stairs',
        'Walking_TO_Descending_stairs',
        'Ascending_stairs_TO_Walking',
        'Descending_stairs_TO_Walking'
    ]

    # 모든 전이에 대해 10%, 20%, 30%, 40% 추가
    mix_pcts = [10, 20, 30, 40]

    for transition in transitions:
        for pct in mix_pcts:
            datasets.append(f"{transition}_{pct}PCT")

    print(f"\nTotal datasets to test: {len(datasets)}")
    print(f"  - Original: 1")
    print(f"  - transitions: {len(transitions) * len(mix_pcts)}")

    all_results = []

    # Run experiments
    for i, dataset_name in enumerate(datasets, 1):
        print(f"\n[Progress: {i}/{len(datasets)}]")
        results = run_experiment(dataset_name, cfg)
        all_results.extend(results)

    # Save all results
    print(f"\n{'='*80}")
    print("SAVING RESULTS")
    print(f"{'='*80}")

    results_dict = {
        'experiment_info': {
            'date': time.strftime('%Y-%m-%d %H:%M:%S'),
            'models': ['GAP', 'TPA', 'Gated-TPA'],
            'total_datasets': len(datasets),
            'datasets': datasets,
            'config': {
                'epochs': cfg.epochs,
                'batch_size': cfg.batch_size,
                'lr': cfg.lr,
                'd_model': cfg.d_model,
                'tpa_num_prototypes': cfg.tpa_num_prototypes,
                'tpa_heads': cfg.tpa_heads
            }
        },
        'results': all_results
    }

    # Save to JSON
    json_path = os.path.join(cfg.save_dir, "pamap2_tpa_transition_cnn.json")
    with open(json_path, 'w') as f:
        json.dump(results_dict, f, indent=2)

    print(f"\nResults saved to: {json_path}")

    # Print summary
    print(f"\n{'='*80}")
    print("SUMMARY")
    print(f"{'='*80}")
    print(f"Total experiments: {len(all_results)}")
    print(f"Total datasets tested: {len(datasets)}")
    print(f"Models compared: 3 (GAP, TPA, Gated-TPA)")

    # Calculate average performance per model
    print(f"\n{'='*80}")
    print("AVERAGE PERFORMANCE (All Datasets)")
    print(f"{'='*80}")

    for model_name in ['GAP', 'TPA', 'Gated-TPA']:
        model_results = [r for r in all_results if r['Model'] == model_name]
        avg_acc = np.mean([r['Test_Accuracy'] for r in model_results])
        avg_f1 = np.mean([r['Test_F1_Score'] for r in model_results])
        print(f"{model_name:12s}: Acc={avg_acc:.4f}, F1={avg_f1:.4f}")

    print(f"\n{'='*80}")
    print("EXPERIMENT COMPLETE")
    print(f"{'='*80}")

Mounted at /content/drive

UNIFIED MODEL COMPARISON: GAP vs TPA vs Gated-TPA

Total datasets to test: 32
  - Original: 1
  - transitions: 32

[Progress: 1/32]

EXPERIMENT: Standing_TO_Walking_10PCT

Loading Standing_TO_Walking_10PCT...
  Path: /content/drive/MyDrive/AI_data/TPA2/pamap2_transition_datasets/Standing_TO_Walking_10PCT
  Train: (34192, 100, 27), Test: (8549, 100, 27)

Dataset splits:
  Train: 27353, Val: 6839, Test: 8549

[Training GAP]
  Epoch  10: Train Acc=0.9441, Val Acc=0.9434, F1=0.9358
  Epoch  20: Train Acc=0.9753, Val Acc=0.9719, F1=0.9695
  Epoch  30: Train Acc=0.9856, Val Acc=0.9820, F1=0.9810
  Epoch  40: Train Acc=0.9901, Val Acc=0.9870, F1=0.9864
  Epoch  50: Train Acc=0.9928, Val Acc=0.9876, F1=0.9868
  Epoch  60: Train Acc=0.9937, Val Acc=0.9896, F1=0.9893
  Epoch  70: Train Acc=0.9950, Val Acc=0.9920, F1=0.9917
  Epoch  80: Train Acc=0.9957, Val Acc=0.9920, F1=0.9916
  Epoch  90: Train Acc=0.9958, Val Acc=0.9924, F1=0.9918
  Epoch 100: Train Acc=0.9963, Val

KeyboardInterrupt: 